# Segmenting and Clustering Neighborhoods in Toronto


## This is a 3 part assignment

## Part 1: Creation of the dataframe from the Wikipedia web page

In [1]:
#Import the relevant packages for the assignment

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


In [2]:
#Scrape the data in Table in the following Wikipedia page and create the dataframe of neighborhoods in Toronto

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Read HTML tables into a ``list`` of ``DataFrame`` objects.
raw_html_tbl_list = pd.read_html(url)

#raw_html_tbl_list is the list of tables read from the wikipedia page
df_neighborhoods = raw_html_tbl_list[0]

#Renamed the column names to match those mentined in the assignment
df_neighborhoods.columns = ['PostalCode','Borough','Neighborhood']

#Retain only the Borough's which have assigned values. Removed the rows with value "Not assigned"
df_neighborhoods = df_neighborhoods[df_neighborhoods['Borough'] != 'Not assigned'].reset_index(drop=True)

#Checked the dataframe. There is no duplicate value for M5A or any other Postal Code. Harbourfront and Regent Park are already meged
df_neighborhoods.duplicated('PostalCode').sum()  

df_neighborhoods.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
#Get the number of rows and columns from the data frame using its shape attribute
df_neighborhoods.shape


(103, 3)

## Part 2: Enrich the data frame with the latitude and longitude.

 Download the csv file and get the geo coordinated for each of the Postal Code

In [4]:
#create a data frame containing the geocodes for the postal codes by reading in the csv
df_geo_coord = pd.read_csv('http://cocl.us/Geospatial_data',index_col=None)

#Rename column Postal Code to PostalCode for consistency and use in the subjsequent merge condition
df_geo_coord.rename(columns=({'Postal Code':'PostalCode'}),inplace=True)
df_geo_coord.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
#df_neighborhoods.join(df_geo_coord, on='PostalCode', how='left'

df_merged = pd.merge(df_neighborhoods,df_geo_coord,on='PostalCode')
df_merged.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Part 3: Explore and cluster the neighborhoods of Toronto


In [6]:
#! conda install -c conda-forge folium
import folium

In [7]:
#Check and confirm that the data frame doesn't have for null cells 
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
Latitude        103 non-null float64
Longitude       103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


In [8]:
#Decided to work only with Boroughs with name Toronto

filter_boroughs_with_Name_Toronto = df_merged['Borough'].str.contains('Toronto')
df_toronto = df_merged[filter_boroughs_with_Name_Toronto].reset_index(drop=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [9]:
#Plot all the 'Toronto named neighborhoods' on the Map
location = [43.6532,-79.3832]
m = folium.Map(location=location,zoom_start=11)

for i in df_toronto.itertuples():
    folium.Marker(location=[i.Latitude,i.Longitude],
            popup=i.Neighborhood,
                  radius = 5,
                  color='blue',
                  fill=True,
                  fill_color='#3186cc',
                  fill_opacity=0.7).add_to(m)
    
m

#### Define Foursquare Credentials and Version

In [10]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: 0GSPO3GEBMTB3KN0J1TRSU1TAGU1LQMAVCNQ5WFKCSZBNIJA
CLIENT_SECRET:DK21CHGLPBCYCNY4OXE5OU0MTOJIDCCXU1J0QUG2WWJ4E03F


First, let's create the GET request URL. Name your URL **url**.

In [11]:
#Define url to explore a location

latitude = 43.6532
longitude = -79.3832

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)


In [12]:
#Testinng the results from a single neighborhood
results = requests.get(url).json()

Define the function to get all the venues for the neighborhood and then also derive the category type of the venue

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Downtown Toronto,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",43.653232,-79.385296
1,Nathan Phillips Square,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",43.652270,-79.383516
2,Indigo,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",43.653515,-79.380696
3,LUSH,"[{'id': '4bf58dd8d48988d10c951735', 'name': 'C...",43.653557,-79.380400
4,Old City Hall,"[{'id': '4bf58dd8d48988d12d941735', 'name': 'M...",43.652009,-79.381744


Define and test the function to get the category type of an individual  venue. 

In [15]:
# function that extracts the category of the venue. This will help test out and optimise the code to determine the venue cataegory
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
# filter the category for each row. 
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Indigo,Bookstore,43.653515,-79.380696
3,LUSH,Cosmetics Shop,43.653557,-79.380400
4,Old City Hall,Monument / Landmark,43.652009,-79.381744
5,CF Toronto Eaton Centre,Shopping Mall,43.654540,-79.380677
6,SEPHORA,Cosmetics Shop,43.653527,-79.380154
7,M Square Coffee Co,Coffee Shop,43.651218,-79.383555
8,Japango,Sushi Restaurant,43.655268,-79.385165
9,Crepe Delicious,Fast Food Restaurant,43.654536,-79.380889


#### Creation of the function to take series of names, geo cordinates(lat and long series) and radius as inputs and return the dataframe containing the relevant details of the associations venues, their coordinates and category. 

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
#Creation of the toronto venues data frame to receive the output from the above defined function

toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

Size of the resulting dataframe

In [19]:
print(toronto_venues.shape)
toronto_venues.head()

(864, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Count of the venues returned for each neighborhood

In [20]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,30,30,30,30,30,30
Christie,16,16,16,16,16,16
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


#### Let's find out how many unique categories can be curated from all the returned venues

In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 191 uniques categories.


#### Analysis of  Each Neighborhood

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
all_columns = list(toronto_onehot.columns.values)
all_columns.remove('Neighborhood')
fixed_columns = ['Neighborhood']+all_columns
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
all_columns

['Airport',
 'Airport Food Court',
 'Airport Lounge',
 'Airport Service',
 'Airport Terminal',
 'American Restaurant',
 'Antique Shop',
 'Aquarium',
 'Art Gallery',
 'Art Museum',
 'Arts & Crafts Store',
 'Asian Restaurant',
 'Auto Workshop',
 'BBQ Joint',
 'Baby Store',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Basketball Stadium',
 'Beer Bar',
 'Beer Store',
 'Belgian Restaurant',
 'Bistro',
 'Board Shop',
 'Boat or Ferry',
 'Bookstore',
 'Boutique',
 'Breakfast Spot',
 'Brewery',
 'Bubble Tea Shop',
 'Burger Joint',
 'Burrito Place',
 'Bus Line',
 'Butcher',
 'Café',
 'Cajun / Creole Restaurant',
 'Candy Store',
 'Caribbean Restaurant',
 'Cheese Shop',
 'Chinese Restaurant',
 'Chocolate Shop',
 'Climbing Gym',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'College Arts Building',
 'College Auditorium',
 'College Gym',
 'College Rec Center',
 'Colombian Restaurant',
 'Comfort Food Restaurant',
 'Comic Shop',
 'Concert Hall',
 'Convenience Store',
 'Cosmetics Shop',
 'Cowork

Size of toronto_onehot dataframe

In [24]:
toronto_onehot.shape

(864, 191)

#### Compute the mean of the frequency of occurrence of each category by grouping rows by neighborhood and by taking the mean

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.071429,0.071429,0.071429,0.214286,0.071429,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.00,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Size of the transformed data frame

In [26]:
toronto_grouped.shape

(39, 191)

#### Print of the top 5 most common venues for each neighborhood

In [27]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Berczy Park----
                venue  freq
0      Farmers Market  0.07
1  Seafood Restaurant  0.07
2         Coffee Shop  0.07
3        Cocktail Bar  0.07
4              Bistro  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1     Coffee Shop  0.09
2  Breakfast Spot  0.09
3     Yoga Studio  0.04
4          Bakery  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.11
1                Garden  0.06
2         Auto Workshop  0.06
3  Fast Food Restaurant  0.06
4               Brewery  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0  Airport Service  0.21
1          Airport  0.07
2    Boat or Ferry  0.07
3  Harbor / Marina  0.07
4      Coffee Shop  0.07


----Central Bay Street----
                venue  freq
0         Coff

#### Put that into a pandas dataframe

In [28]:
#funtion to sort venues in dexcending order 

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
#Dta frame o get the top venues for each of the Neighborhoods

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Coffee Shop,Farmers Market,Seafood Restaurant,Jazz Club,Basketball Stadium,Beer Bar,Bistro,Breakfast Spot,Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Yoga Studio,Performing Arts Venue,Burrito Place,Restaurant,Climbing Gym,Nightclub,Bar
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Fast Food Restaurant,Recording Studio,Butcher,Restaurant,Spa,Auto Workshop,Burrito Place,Pizza Place,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport,Harbor / Marina,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Food Court
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Park,Dessert Shop,Spa,Modern European Restaurant,Seafood Restaurant,Sandwich Place,Japanese Restaurant


## Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [30]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 3, 3, 0, 0, 3, 0, 3, 3], dtype=int32)

New dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [31]:
# add clustering labels at the first column
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Chocolate Shop,Dessert Shop,Distribution Center,Restaurant,Spa
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Coffee Shop,Diner,Yoga Studio,Sushi Restaurant,College Auditorium,Park,Mexican Restaurant,Creperie,Café,Burrito Place
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Café,Clothing Store,Theater,Tea Room,Gastropub,Pizza Place,Japanese Restaurant,Sandwich Place,Burger Joint,Shopping Mall
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Restaurant,Japanese Restaurant,Café,Coffee Shop,Gastropub,Cosmetics Shop,Diner,Italian Restaurant,Ice Cream Shop,Farmers Market
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Trail,Health Food Store,Pub,Yoga Studio,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


Visulization of the Clusters

In [32]:
# create map
toronto_location = [43.6532,-79.3832]
map_clusters = folium.Map(location=toronto_location, zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


#### Examine each of the Clusters

Cluster 1

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Café,Clothing Store,Theater,Tea Room,Gastropub,Pizza Place,Japanese Restaurant,Sandwich Place,Burger Joint,Shopping Mall
3,Downtown Toronto,0,Restaurant,Japanese Restaurant,Café,Coffee Shop,Gastropub,Cosmetics Shop,Diner,Italian Restaurant,Ice Cream Shop,Farmers Market
6,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Park,Dessert Shop,Spa,Modern European Restaurant,Seafood Restaurant,Sandwich Place,Japanese Restaurant
7,Downtown Toronto,0,Grocery Store,Café,Park,Candy Store,Baby Store,Nightclub,Coffee Shop,Diner,Italian Restaurant,Restaurant
8,Downtown Toronto,0,Coffee Shop,Café,Hotel,Pizza Place,Monument / Landmark,Gym / Fitness Center,Salon / Barbershop,Juice Bar,Seafood Restaurant,Smoke Shop
13,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Gym / Fitness Center,Sandwich Place,Bookstore,Pub,Beer Bar,Hotel
14,West Toronto,0,Café,Breakfast Spot,Coffee Shop,Yoga Studio,Performing Arts Venue,Burrito Place,Restaurant,Climbing Gym,Nightclub,Bar
16,Downtown Toronto,0,Café,Gastropub,Coffee Shop,Restaurant,Hotel,Steakhouse,Museum,Deli / Bodega,Pub,Japanese Restaurant
17,East Toronto,0,Café,Coffee Shop,Bakery,Seafood Restaurant,Sandwich Place,Brewery,Cheese Shop,Pet Store,Park,Yoga Studio
24,Central Toronto,0,Café,Sandwich Place,Coffee Shop,Burger Joint,Indian Restaurant,Pub,History Museum,Liquor Store,Middle Eastern Restaurant,BBQ Joint


Cluster 2

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1,Park,Swim School,Bus Line,Yoga Studio,Dance Studio,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
29,Central Toronto,1,Park,Restaurant,Lawyer,Trail,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
33,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


Cluster 3

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,2,Jewelry Store,Mexican Restaurant,Trail,Sushi Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


Cluster 4

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,3,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Chocolate Shop,Dessert Shop,Distribution Center,Restaurant,Spa
1,Downtown Toronto,3,Coffee Shop,Diner,Yoga Studio,Sushi Restaurant,College Auditorium,Park,Mexican Restaurant,Creperie,Café,Burrito Place
4,East Toronto,3,Trail,Health Food Store,Pub,Yoga Studio,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
5,Downtown Toronto,3,Cocktail Bar,Coffee Shop,Farmers Market,Seafood Restaurant,Jazz Club,Basketball Stadium,Beer Bar,Bistro,Breakfast Spot,Restaurant
9,West Toronto,3,Pharmacy,Bakery,Pet Store,Supermarket,Park,Middle Eastern Restaurant,Recording Studio,Café,Brewery,Bar
10,Downtown Toronto,3,Park,Café,Hotel,Plaza,Dessert Shop,Lake,Japanese Restaurant,Skating Rink,Roof Deck,Ice Cream Shop
11,West Toronto,3,Bar,Men's Store,Vietnamese Restaurant,Asian Restaurant,Yoga Studio,Bistro,Brewery,Record Shop,Pizza Place,Cocktail Bar
12,East Toronto,3,Greek Restaurant,Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Bakery,Brewery,Juice Bar,Bookstore,Spa
15,East Toronto,3,Park,Sushi Restaurant,Pet Store,Pizza Place,Pub,Liquor Store,Restaurant,Burrito Place,Sandwich Place,Brewery
20,Central Toronto,3,Hotel,Gym / Fitness Center,Breakfast Spot,Sandwich Place,Department Store,Food & Drink Shop,Park,Pizza Place,Comfort Food Restaurant,Dance Studio


Cluster 5

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Garden,Home Service,Yoga Studio,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
